In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Design Model

<img src="https://markdown-yqguo.oss-cn-beijing.aliyuncs.com/markdown-yqguo/v2-020158fdd8894772d43199e0cce9d3a8_r.jpg" alt="preview" style="zoom: 80%;" />

- nn.Conv2d和nn.Linear后面要接上nn.functional.relu() 或 nn.ReLU()  
- nn.functional.dropout(p=0.5)或nn.Dropout(p=0.5)

In [27]:
class _cpLayer(nn.Module):
    def __init__(self,conv_num,num_input_features,num_output_features):
        super(_cpLayer,self).__init__()
        layer=[]
        for i in range(conv_num):
            #same卷积
            layer.append(nn.Conv2d(in_channels=num_input_features,out_channels=num_output_features,kernel_size=3,padding=1,stride=1))
            num_input_features=num_output_features
            layer.append(nn.ReLU())
        layer.append(nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer = nn.Sequential(*layer)
        
    def forward(self,x):
        return self.layer(x)
    
    
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.cp1=_cpLayer(2,3,64)
        self.cp2=_cpLayer(2,64,128)
        self.cp3=_cpLayer(3,128,256)
        self.cp4=_cpLayer(3,256,512)
        self.cp5=_cpLayer(3,512,512)
        self.fc1=nn.Linear(7*7*512,4096)
        self.fc2=nn.Linear(4096,4096)
        self.fc3=nn.Linear(4096,1000) #分成1000类的
        
    def forward(self,x):
        in_size=x.size(0)
        x=self.cp1(x)
        x=self.cp2(x)
        x=self.cp3(x)
        x=self.cp4(x)
        x=self.cp5(x)
        x=x.view(in_size,-1)
        x=F.dropout(F.relu(self.fc1(x)),p=0.5)
        x=F.dropout(F.relu(self.fc2(x)),p=0.5)
        x=self.fc3(x)
        
        return x

In [29]:
net=Net()
x=torch.rand(5,3,224,224)
out=net(x)
print(out.size())

torch.Size([5, 1000])
